In [1]:
import numpy as np
from pyscf import gto, lib
from pyscf.gto.eval_gto import make_screen_index, _GTO_EVAL_FUNCTIONS

In [2]:
_GTO_EVAL_FUNCTIONS["GTOval_ipr"] = (9, 9)
_GTO_EVAL_FUNCTIONS["GTOval_iprc"] = (9, 9)

## Definition

In [3]:
mol = gto.Mole(
    atom="""
        C          0.99590        0.00874        0.02912
        C          2.51497        0.01491        0.04092
        C          3.05233        0.96529        1.10755
        C          4.57887        0.97424        1.12090
        C          5.10652        1.92605        2.19141
        C          6.63201        1.93944        2.20819
        C          7.15566        2.89075        3.28062
        C          8.68124        2.90423        3.29701
        C          9.20897        3.85356        4.36970
        C         10.73527        3.86292        4.38316
        C         11.27347        4.81020        5.45174
        C         12.79282        4.81703        5.46246
        H          0.62420       -0.67624       -0.73886
        H          0.60223        1.00743       -0.18569
        H          0.59837       -0.31563        0.99622
        H          2.88337        0.31565       -0.94674
        H          2.87961       -1.00160        0.22902
        H          2.67826        0.66303        2.09347
        H          2.68091        1.98005        0.91889
        H          4.95608        1.27969        0.13728
        H          4.95349       -0.03891        1.31112
        H          4.72996        1.62033        3.17524
        H          4.73142        2.93925        2.00202
        H          7.00963        2.24697        1.22537
        H          7.00844        0.92672        2.39728
        H          6.77826        2.58280        4.26344
        H          6.77905        3.90354        3.09209
        H          9.05732        3.21220        2.31361
        H          9.05648        1.89051        3.48373
        H          8.83233        3.54509        5.35255
        H          8.83406        4.86718        4.18229
        H         11.10909        4.16750        3.39797
        H         11.10701        2.84786        4.56911
        H         10.90576        4.50686        6.43902
        H         10.90834        5.82716        5.26681
        H         13.18649        3.81699        5.67312
        H         13.16432        5.49863        6.23161
        H         13.18931        5.14445        4.49536
    """, basis="def2-QZVP").build()

In [4]:
x = np.arange(2048)
coords_2 = np.stack([np.sin(x), np.cos(x), np.sin(x + 0.5)], axis=1)

In [5]:
x = np.arange(131072)
coords_128 = np.stack([np.sin(x), np.cos(x), np.sin(x + 0.5)], axis=1)

In [6]:
x = np.arange(1048576)
coords_1k = np.stack([np.sin(x), np.cos(x), np.sin(x + 0.5)], axis=1)

In [7]:
shls_slice = [sh0, sh1] = [4, 432]

## Usual case

In [8]:
ao = mol.eval_gto("GTOval_sph", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(378.9344483361458, [2048, 1464])

In [9]:
ao = mol.eval_gto("GTOval_cart", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(198.05403491229913, [2048, 1774])

In [10]:
ao = mol.eval_gto("GTOval_sph_deriv1", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(1158.9931602933664, [2048, 1464, 4])

In [11]:
ao = mol.eval_gto("GTOval_cart_deriv1", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(-1725.5519928866993, [2048, 1774, 4])

In [12]:
ao = mol.eval_gto("GTOval_sph_deriv2", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(2777.1107970453154, [2048, 1464, 10])

In [13]:
ao = mol.eval_gto("GTOval_cart_deriv2", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(718.8381138263526, [2048, 1774, 10])

In [14]:
ao = mol.eval_gto("GTOval_sph_deriv3", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(5879.015564180246, [2048, 1464, 20])

In [15]:
ao = mol.eval_gto("GTOval_cart_deriv3", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(-2664.5459224494057, [2048, 1774, 20])

In [16]:
ao = mol.eval_gto("GTOval_sph_deriv4", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(30771.778920777477, [2048, 1464, 35])

In [17]:
ao = mol.eval_gto("GTOval_cart_deriv4", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(33427.10951556449, [2048, 1774, 35])

## With args

In [18]:
ao = mol.eval_gto("GTOval_sph", coords_2, cutoff=1e-2).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "sph")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(-86.33740170264781, [2048, 1358])

In [19]:
ao = mol.eval_gto("GTOval_sph", coords_2, cutoff=1e-8).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "sph")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(-89.26220691313141, [2048, 1358])

In [20]:
ao = mol.eval_gto("GTOval_cart", coords_2, cutoff=1e-5).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "cart")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(143.06831738790123, [2048, 1649])

In [21]:
ao = mol.eval_gto("GTOval_sph_deriv1", coords_2, cutoff=1e-2).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "sph")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(-926.9693102995727, [2048, 1358, 4])

In [22]:
ao = mol.eval_gto("GTOval_cart_deriv1", coords_2, cutoff=1e-2).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "cart")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(-2165.767335096756, [2048, 1649, 4])

In [23]:
ao = mol.eval_gto("GTOval_sph_deriv2", coords_2, cutoff=1e-2).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "sph")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(-427.9248897536187, [2048, 1358, 10])

In [24]:
ao = mol.eval_gto("GTOval_cart_deriv2", coords_2, cutoff=1e-2).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "cart")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(-2661.780110374988, [2048, 1649, 10])

In [25]:
ao = mol.eval_gto("GTOval_sph_deriv3", coords_2, cutoff=1e-2).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "sph")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(-4097.951202338538, [2048, 1358, 20])

In [26]:
ao = mol.eval_gto("GTOval_cart_deriv3", coords_2, cutoff=1e-2).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "cart")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(-2321.4395429846345, [2048, 1649, 20])

In [27]:
ao = mol.eval_gto("GTOval_sph_deriv4", coords_2, cutoff=1e-2).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "sph")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(-974.7870302755828, [2048, 1358, 35])

In [28]:
ao = mol.eval_gto("GTOval_cart_deriv4", coords_2, cutoff=1e-2).swapaxes(-1, -2)
ao_loc = gto.moleintor.make_loc(mol._bas, "cart")
ao_slc = slice(ao_loc[sh0], ao_loc[sh1])
ao = ao[..., ao_slc, :]
float(lib.fp(ao)), list(ao.shape[::-1])

(-31806.509015190735, [2048, 1649, 35])

## Other real properties

In [29]:
ao = mol.eval_gto("GTOval_ip_sph", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(331.80038695474656, [2048, 1464, 3])

In [30]:
ao = mol.eval_gto("GTOval_ig_sph", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(-751.1976925410391, [2048, 1464, 3])

In [31]:
ao = mol.eval_gto("GTOval_ipig_sph", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(-3052.9189303084386, [2048, 1464, 9])

In [32]:
ao = mol.eval_gto("GTOval_ipr_sph", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(54.167001057908806, [2048, 1464, 9])

In [33]:
ao = mol.eval_gto("GTOval_iprc_sph", coords_2).swapaxes(-1, -2)
float(lib.fp(ao)), list(ao.shape[::-1])

(-901.1885030079377, [2048, 1464, 9])